# **Definición de Endpoints para Análisis de Datos de Steam Games**

Para nuestra API, desarrollaremos funciones especializadas para los siguientes endpoints:

1. *@app.get('/playtime_genre')*
**Función PlayTimeGenre(genero: str):** Retorna el año con más horas jugadas para un género.

2. *@app.get('/user_for_genre')*
**Función UserForGenre(genero: str):** Devuelve el usuario con más horas en un género y su distribución anual de horas jugadas.

3. *@app.get('/users_recommend')*
**Función UsersRecommend(año: int):** Lista el top 3 de juegos más recomendados en un año.

4. *@app.get('/users_worst_developer')*
**Función UsersWorstDeveloper(año: int):** Muestra las 3 desarrolladoras con menos recomendaciones en un año.

5. *@app.get('/sentiment_analysis')*
**Función SentimentAnalysis(empresa: str):** Presenta un análisis de sentimiento de reseñas por desarrolladora.
Cada función aprovechará nuestra base de datos previamente limpiada para generar insights significativos.

In [1]:
#Importamos las librerias
import pandas as pd
import numpy as np 
import pyarrow as pa
import pyarrow.parquet as pq
from fastapi import FastAPI
import unicodedata
app = FastAPI()

In [2]:
pip install uvicorn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


**Extracción de los datos**

In [3]:
# Carga de datos de juegos de Steam
steam_games = pd.read_csv('data/steam_games_cleaned.csv')

# Carga de datos de reseñas de usuarios
user_reviews = pd.read_csv('data/user_reviews_cleaned.csv')

# Carga de datos de artículos de usuarios
user_items = pd.read_parquet('data/user_items_cleaned.parquet')

## 1. Función **def PlayTimeGenre**(genero:str):

Para analizar las preferencias de juego, utilizaremos la tabla steam_games para el género y año de lanzamiento, y la tabla user_items para las horas jugadas (playtime_forever). Estos datos nos ayudarán a identificar tendencias y patrones de uso.

**Crear una tabla con las columnas que necesitamos para obtener los datos que necesitamos**

In [4]:
#Revisar la tabla steam_games
steam_games.head(3)

,item_id,app_name,genres,year_of_release,publisher,developer
0,761140.0,Lost Summoner Kitty,Action,2018.0,Kotoshiro,Kotoshiro
1,761140.0,Lost Summoner Kitty,Casual,2018.0,Kotoshiro,Kotoshiro
2,761140.0,Lost Summoner Kitty,Indie,2018.0,Kotoshiro,Kotoshiro


In [5]:
#Extraer las columnas necesarias para esta función
genre= steam_games[["item_id","genres","year_of_release"]]

In [6]:
#Revisar la tabla user_items
user_items.head(3)

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,10,Counter-Strike,6,0
1,76561197970982479,277,20,Team Fortress Classic,0,0
2,76561197970982479,277,30,Day of Defeat,7,0


In [7]:
#Seleccionar las columnas que necesito
play_time= user_items[["item_id","playtime_forever"]]

In [8]:
# Convertir 'item_id' a float64 en ambos DataFrames
play_time['item_id'] = play_time['item_id'].astype('float64')
genre['item_id'] = genre['item_id'].astype('float64')

# Realizar el merge
genre_funcion = play_time.merge(genre, on="item_id")

# Convertir 'item_id' a entero en el DataFrame resultante
genre_funcion['item_id'] = genre_funcion['item_id'].astype('int64')

# Si 'year_of_release' está en el DataFrame y deseas convertirlo también a entero
genre_funcion['year_of_release'] = genre_funcion['year_of_release'].fillna(0).astype('int64')

# Revisar la nueva tabla
genre_funcion.head(3)

C:\Users\USER\AppData\Local\Temp\ipykernel_12996\3377124161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  play_time['item_id'] = play_time['item_id'].astype('float64')
C:\Users\USER\AppData\Local\Temp\ipykernel_12996\3377124161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre['item_id'] = genre['item_id'].astype('float64')


,item_id,playtime_forever,genres,year_of_release
0,10,6,Action,2000
1,10,0,Action,2000
2,10,0,Action,2000


**Obtener los datos que necesitamos para nuestra función**

In [9]:
# Agrupamos los datos por género y año de lanzamiento y sumamos las horas jugadas
total_playtime_by_year_genre = genre_funcion.groupby(["genres", "year_of_release"]).playtime_forever.sum().reset_index(name='total_playtime')

# Ordenamos los resultados por el total de horas jugadas de manera descendente
sorted_playtime = total_playtime_by_year_genre.sort_values(by="total_playtime", ascending=False)

# Muestra los resultados
sorted_playtime

,genres,year_of_release,total_playtime
26,Action,2012,1085635110
158,Indie,2006,446115272
257,Simulation,2006,444536391
27,Action,2013,415522232
25,Action,2011,331697668
...,...,...,...
123,Education,2010,9
239,Simulation,1988,1
271,Software Training,2010,0
190,RPG,1991,0


**Definir el Máximo de Horas Jugadas por Género de Videojuegos**

In [10]:
# Agrupamos por género y calculamos el máximo de horas jugadas
max_playtime_by_genre = sorted_playtime.groupby("genres")["total_playtime"].agg(MaxHours="max").reset_index()

# Mostramos los primeros registros del resultado
max_playtime_by_genre.head()

,genres,MaxHours
0,Action,1085635110
1,Adventure,221707756
2,Animation &amp; Modeling,1345545
3,Audio Production,455463
4,Casual,81708365


**Encontrar el maximo de horas jugadas por género**

In [11]:
# Agrupamos los datos por género y calculamos el máximo de horas jugadas para cada género
max_hours_per_genre = sorted_playtime.groupby("genres")["total_playtime"].agg(MaxHours="max").reset_index()

# Mostramos los primeros cinco registros para obtener una vista previa del resultado
max_hours_per_genre.head()

,genres,MaxHours
0,Action,1085635110
1,Adventure,221707756
2,Animation &amp; Modeling,1345545
3,Audio Production,455463
4,Casual,81708365


In [12]:
# Obtenemos las dimensiones del DataFrame
max_hours_per_genre.shape

(21, 2)

**Renombramos la columna para facilitar el join con max_hours_per_genre. Esto asegura que las columnas tengan nombres consistentes para el merge.

In [13]:
# Renombrar columna para el merge
genre_playtime = sorted_playtime.rename(columns={'total_playtime': 'MaxHours'})

# Mostrar DataFrame actualizado
genre_playtime

,genres,year_of_release,MaxHours
26,Action,2012,1085635110
158,Indie,2006,446115272
257,Simulation,2006,444536391
27,Action,2013,415522232
25,Action,2011,331697668
...,...,...,...
123,Education,2010,9
239,Simulation,1988,1
271,Software Training,2010,0
190,RPG,1991,0


In [14]:
# Realizamos un merge entre max_hours_per_genre y genre_playtime
merged_genre_data = pd.merge(max_hours_per_genre, genre_playtime)

# Mostramos el DataFrame resultante
merged_genre_data

,genres,MaxHours,year_of_release
0,Action,1085635110,2012
1,Adventure,221707756,2011
2,Animation &amp; Modeling,1345545,2015
3,Audio Production,455463,2014
4,Casual,81708365,2015
5,Design &amp; Illustration,1930339,2012
6,Early Access,118902893,2013
7,Education,340567,2013
8,Free to Play,146213734,2013
9,Indie,446115272,2006


In [15]:
# Guarda el DataFrame 'tabla_final' en un archivo CSV
merged_genre_data_2 = "funcion_1.csv"
merged_genre_data.to_csv(merged_genre_data_2, index=False, encoding="utf-8")

In [16]:
# Convertir un archivo CSV a Parquet

# Lee el archivo CSV
funcion_1 = pd.read_csv("funcion_1.csv")

# Define el nombre y la ubicación del archivo Parquet
genre_file = "data/funcion_1.parquet"

# Convierte el DataFrame a una tabla de PyArrow y guarda en formato Parquet
table = pa.Table.from_pandas(funcion_1)
pq.write_table(table, genre_file)

In [17]:
@app.get('/PlayTimeGenre/{genero}')
def PlayTimeGenre(genero):
    # Convertimos tanto el género de entrada como la columna del DataFrame a minúsculas
    genero_min = genero.lower()
    merged_genre_data["genres"] = merged_genre_data["genres"].str.lower()

    # Filtra los datos para el género especificado
    genre_data = merged_genre_data[merged_genre_data["genres"] == genero_min]
    
    # Encuentra el año con más horas jugadas
    if not genre_data.empty:
        max_playtime_row = genre_data.sort_values(by='MaxHours', ascending=False).iloc[0]

        # Devuelve el año de lanzamiento con más horas jugadas para el género dado
        return {"Año de lanzamiento con más horas jugadas para Género " + genero: int(max_playtime_row["year_of_release"])}
    else:
        return {"Error": "Género no encontrado o sin datos"}

In [18]:
PlayTimeGenre('Indie')

{'Año de lanzamiento con más horas jugadas para Género Indie': 2006}

## 2. Función **def UserForGenre(genero:str)**:

En esta función, avanzamos desde la Función 1, utilizando nuestra tabla que integra datos de Steam e Items. Ahora añadimos el user_id, que originalmente está en formato de objeto. Convertimos user_id a formato de texto (string) para mejorar la consistencia y precisión del análisis. Este paso es esencial para asegurar la compatibilidad y la integridad de los datos en todas las etapas del proceso.

In [19]:
# Convertir 'user_id' de object a string.
user_items['user_id'] = user_items['user_id'].astype('string')

# Seleccionar columnas relevantes de 'user_items'.
play_time_user = user_items[["item_id", "playtime_forever", "user_id"]]

# Convertir 'item_id' a float64 para manejar NaN y luego a int64.
play_time_user['item_id'] = play_time_user['item_id'].astype('float64').fillna(0).astype('int64')

# Convertir 'item_id' en 'genre' a float64 y luego a int64.
genre['item_id'] = genre['item_id'].astype('float64').fillna(0).astype('int64')

# Merge de 'play_time_user' con 'genre' usando 'item_id'.
combined_genre_user = play_time_user.merge(genre, on="item_id")

# Convertir 'genres' y 'user_id' a string.
combined_genre_user['genres'] = combined_genre_user['genres'].astype('string')
combined_genre_user['user_id'] = combined_genre_user['user_id'].astype('string')

# Reemplazar NaN en 'year_of_release' y convertir a int64 para eliminar la parte decimal.
combined_genre_user['year_of_release'] = combined_genre_user['year_of_release'].fillna(0).astype('int64')

# Visualizar las primeras 5 filas del DataFrame combinado para verificar los cambios.
combined_genre_user.head(5)


C:\Users\USER\AppData\Local\Temp\ipykernel_12996\117745472.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  play_time_user['item_id'] = play_time_user['item_id'].astype('float64').fillna(0).astype('int64')
C:\Users\USER\AppData\Local\Temp\ipykernel_12996\117745472.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre['item_id'] = genre['item_id'].astype('float64').fillna(0).astype('int64')


,item_id,playtime_forever,user_id,genres,year_of_release
0,10,6,76561197970982479,Action,2000
1,10,0,js41637,Action,2000
2,10,0,Riot-Punch,Action,2000
3,10,93,doctr,Action,2000
4,10,108,corrupted_soul,Action,2000


In [20]:
# Agrupar por género, usuario y año de lanzamiento, y sumar las horas jugadas
total_playtime_by_user_year = combined_genre_user.groupby(["genres", "user_id", "year_of_release"])["playtime_forever"].sum().reset_index()

# Visualizar el DataFrame resultante
total_playtime_by_user_year

,genres,user_id,year_of_release,playtime_forever
0,Action,--000--,2009,5329
1,Action,--000--,2010,22
2,Action,--000--,2011,6522
3,Action,--000--,2012,109346
4,Action,--000--,2013,363
...,...,...,...,...
3440985,Web Publishing,zepavil,2015,9010
3440986,Web Publishing,zeshirky,2007,1
3440987,Web Publishing,zevlupine,2012,4
3440988,Web Publishing,zilaman,2013,9


Ahora vamos a identificar los patrones de juego de los usuarios en relación con diferentes géneros de videojuegos. El objetivo es determinar cuáles usuarios dedican más tiempo a cada género y cómo se distribuye ese tiempo a lo largo de los años.

In [21]:
# Agrupar por género y usuario para sumar las horas jugadas
total_hours_by_user = combined_genre_user.groupby(["genres", "user_id"])["playtime_forever"].sum().reset_index()
total_hours_by_user.head()  # Vista previa de los datos agregados por usuario


,genres,user_id,playtime_forever
0,Action,--000--,139469
1,Action,--ace--,69325
2,Action,--ionex--,38315
3,Action,-2SV-vuLB-Kg,42500
4,Action,-404PageNotFound-,117423


In [22]:
# Encontrar el usuario con la máxima cantidad de horas jugadas por género
max_hours_by_genre = total_hours_by_user.groupby("genres")["playtime_forever"].agg(playtime_forever="max").reset_index()
max_hours_by_genre.head()

,genres,playtime_forever
0,Action,1699307
1,Adventure,2191551
2,Animation &amp; Modeling,168314
3,Audio Production,109916
4,Casual,1224933


In [23]:
# Combinar para obtener el detalle del usuario con más horas jugadas por género
detailed_max_hours = pd.merge(max_hours_by_genre, total_hours_by_user)
detailed_max_hours

,genres,playtime_forever,user_id
0,Action,1699307,Sp3ctre
1,Adventure,2191551,REBAS_AS_F-T
2,Animation &amp; Modeling,168314,ScottyG555
3,Audio Production,109916,Lickidactyl
4,Casual,1224933,REBAS_AS_F-T
5,Design &amp; Illustration,168314,ScottyG555
6,Early Access,316969,76561197978756659
7,Education,65427,76561198059330972
8,Free to Play,808241,idonothack
9,Indie,2402994,REBAS_AS_F-T


In [24]:
# Preparar para agregar el total de horas jugadas por año
playtime_per_year = total_playtime_by_user_year.rename(columns={"playtime_forever": "playtime_forever_year"})
playtime_per_year.head(5)  # Vista previa de los datos con horas jugadas por año

,genres,user_id,year_of_release,playtime_forever_year
0,Action,--000--,2009,5329
1,Action,--000--,2010,22
2,Action,--000--,2011,6522
3,Action,--000--,2012,109346
4,Action,--000--,2013,363


In [25]:
# Verificar el cruce de datos para el usuario específico 'ScottyG555' en el género 'Video Production'
test_user_playtime = playtime_per_year[(playtime_per_year['user_id'] == 'ScottyG555') & (playtime_per_year['genres'] == 'Video Production')]

# Visualizar los resultados de la prueba
test_user_playtime

,genres,user_id,year_of_release,playtime_forever_year
3435530,Video Production,ScottyG555,2015,168314


In [26]:
# Combinar tabla_user con playtime_per_year para mayor detalle
combined_user_data = pd.merge(detailed_max_hours, playtime_per_year)
combined_user_data

,genres,playtime_forever,user_id,year_of_release,playtime_forever_year
0,Action,1699307,Sp3ctre,1993,0
1,Action,1699307,Sp3ctre,1995,217
2,Action,1699307,Sp3ctre,1996,0
3,Action,1699307,Sp3ctre,1998,0
4,Action,1699307,Sp3ctre,1999,44
...,...,...,...,...,...
163,Utilities,207651,76561198073642113,2014,207651
164,Video Production,168314,ScottyG555,2015,168314
165,Web Publishing,142964,Xyphien,2005,7296
166,Web Publishing,142964,Xyphien,2012,64657


In [27]:

# Verificar los registros para 'ScottyG555' en el género 'Video Production'
test_combined_data = combined_user_data[(combined_user_data['user_id'] == 'ScottyG555') & (combined_user_data['genres'] == 'Video Production')]
test_combined_shape = test_combined_data.shape  # Obtener la cantidad de registros

# Resultado de la verificación, tiene que dar el resultado 7
test_combined_shape

(1, 5)

Ajustamos y combinamos datasets para analizar las horas de juego por género y usuario. Verificación de Datos: Confirmamos la precisión de los datos para la creación de funciones.

In [28]:
# Renombrar columnas y preparar el DataFrame
combined_user_playtime = combined_user_data.rename(columns={'year_of_release': 'Year', 'playtime_forever_year': 'Hours Played'})

# Convertir 'Year' a numérico, tratando los valores no numéricos como NaN
combined_user_playtime["Year"] = pd.to_numeric(combined_user_playtime["Year"], errors='coerce')

# Convertir 'Year' a entero (Int64 para manejar NaN)
combined_user_playtime["Year"] = combined_user_playtime["Year"].astype('Int64')

# Visualizar el DataFrame actualizado
combined_user_playtime

,genres,playtime_forever,user_id,Year,Hours Played
0,Action,1699307,Sp3ctre,1993,0
1,Action,1699307,Sp3ctre,1995,217
2,Action,1699307,Sp3ctre,1996,0
3,Action,1699307,Sp3ctre,1998,0
4,Action,1699307,Sp3ctre,1999,44
...,...,...,...,...,...
163,Utilities,207651,76561198073642113,2014,207651
164,Video Production,168314,ScottyG555,2015,168314
165,Web Publishing,142964,Xyphien,2005,7296
166,Web Publishing,142964,Xyphien,2012,64657


**Guardamos la dataframe en .parquet**

In [29]:
# Guardar el DataFrame en un archivo Parquet
combined_user_playtime.to_parquet('data/funcion_2.parquet')

**Creamos la función**

In [30]:
# Definimos una función auxiliar para normalizar los strings, eliminando tildes y convirtiendo a minúsculas
import unicodedata

def normalize_string(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn').lower()

In [31]:
@app.get('/UserForGenre/{genero}')
def UserForGenre(genero):
    # Normalizar el género de entrada
    genero_norm = normalize_string(genero)

    # Normalizar la columna de géneros en el DataFrame y buscar el usuario con más horas jugadas
    combined_user_playtime['genres_norm'] = combined_user_playtime['genres'].apply(normalize_string)
    top_user = combined_user_playtime[combined_user_playtime['genres_norm'] == genero_norm]["user_id"].iloc[0]

    # Filtrar los datos para el usuario y género seleccionado
    user_history = combined_user_playtime[(combined_user_playtime['user_id'] == top_user) & (combined_user_playtime['genres_norm'] == genero_norm)]

    # Seleccionar solo las columnas necesarias y convertir a diccionario
    user_history_filtered = user_history[['Year', 'Hours Played']].copy()
    user_history_dict = user_history_filtered.to_dict(orient="records")

    # Devolver los resultados
    return {"Usuario": top_user, "con más horas jugadas para": genero, "Historial acumulado": user_history_dict}



In [32]:
UserForGenre('indie')

{'Usuario': 'REBAS_AS_F-T',
 'con más horas jugadas para': 'indie',
 'Historial acumulado': [{'Year': 1999, 'Hours Played': 0},
  {'Year': 2001, 'Hours Played': 11},
  {'Year': 2003, 'Hours Played': 1863},
  {'Year': 2005, 'Hours Played': 0},
  {'Year': 2006, 'Hours Played': 1673},
  {'Year': 2007, 'Hours Played': 1070},
  {'Year': 2008, 'Hours Played': 1366},
  {'Year': 2009, 'Hours Played': 28993},
  {'Year': 2010, 'Hours Played': 21487},
  {'Year': 2011, 'Hours Played': 100155},
  {'Year': 2012, 'Hours Played': 148459},
  {'Year': 2013, 'Hours Played': 169349},
  {'Year': 2014, 'Hours Played': 326927},
  {'Year': 2015, 'Hours Played': 751765},
  {'Year': 2016, 'Hours Played': 815989},
  {'Year': 2017, 'Hours Played': 33887}]}

## 3. Función **defUsersRecommend(año:int)**:


**Crear una base de datos más ligera para trabajar esta función**

In [33]:
#Seleccionamos solo las columnas necesarias de cada DataFrame y realiza un merge
light_df = pd.merge(
    user_reviews[['reviews_year', 'reviews_item_id', 'reviews_recommend', 'sentiment_analysis']],
    steam_games[['item_id', 'app_name']].drop_duplicates(subset='item_id'),
    left_on='reviews_item_id', right_on='item_id'
)

In [34]:
#Revisamos tipos de datos
light_df.dtypes

reviews_year           object
reviews_item_id       float64
reviews_recommend      object
sentiment_analysis      int64
item_id               float64
app_name               object
dtype: object

In [35]:
# Reemplazar el texto 'Dato no disponible' con NaN o un valor numérico
# Aquí estoy usando 0, pero puedes elegir otro valor si es más apropiado
light_df['reviews_year'] = light_df['reviews_year'].replace('Dato no disponible', 0)

# Ahora que todos los valores son numéricos o NaN, convertir a int
light_df['reviews_year'] = light_df['reviews_year'].astype('int')


In [36]:
# Paso 2: Guardar los DataFrames ligeros en formato Parquet
light_df.to_parquet('data/funcion_3.parquet')

In [37]:
@app.get('/UsersRecommend')
def UsersRecommend(year: int):
    """
    Devuelve los 3 juegos más recomendados por usuarios para un año dado.
    Considera solo recomendaciones donde 'reviews.recommend' es True y el análisis de sentimiento es positivo o neutral.

    Parámetros:
    year (int): El año para filtrar las reseñas de usuario.

    Devuelve:
    list: Una lista de los 3 juegos más recomendados para el año dado.
    """
    # Filtrar reseñas por el año dado, recomendaciones y análisis de sentimiento positivo o neutral
    filtered_reviews = light_df[
        (light_df['reviews_year'] == year) & 
        (light_df['reviews_recommend'] == True) & 
        (light_df['sentiment_analysis'].isin([1, 2]))  # 1 para positivo, 2 para neutral
    ]

    # Contar recomendaciones por ID del juego
    recommend_count = filtered_reviews.groupby('reviews_item_id').size().reset_index(name='recommend_count')

    # Ordenar y obtener los 3 juegos principales
    top_3_games = recommend_count.sort_values(by='recommend_count', ascending=False).head(3)

    # Combinar con el dataset de juegos para obtener nombres
    top_3_games_with_names = top_3_games.merge(
        light_df[['item_id', 'app_name']].drop_duplicates('item_id'),
        left_on='reviews_item_id',
        right_on='item_id'
    )

    # Preparar el resultado final
    top_3_games_list = top_3_games_with_names[['app_name', 'recommend_count']].to_dict(orient='records')
    top_3_result = []
    for i in range(min(3, len(top_3_games_list))):
        game_info = {
            "Juego": top_3_games_list[i]['app_name'],
            "Recomendaciones": top_3_games_list[i]['recommend_count']
        }
        top_3_result.append(game_info)

    if not top_3_result:
        return f"No hay suficientes datos para juegos recomendados en el año {year}"
    else:
        return top_3_result


In [38]:
UsersRecommend(2014)

[{'Juego': 'Team Fortress 2', 'Recomendaciones': 1146},
 {'Juego': 'Counter-Strike: Global Offensive', 'Recomendaciones': 794},
 {'Juego': "Garry's Mod", 'Recomendaciones': 509}]

## 4. Funcion **def UsersWorstDeveloper( año : int )**

In [39]:
# Paso 1: Reemplazar 'Dato no disponible' y otros valores no numéricos con NaN o 0
user_reviews['reviews_year'] = pd.to_numeric(user_reviews['reviews_year'], errors='coerce').fillna(0)

In [40]:
# Paso 2: Convertir 'reviews_year' a int ahora que todos los valores son numéricos
user_reviews['reviews_year'] = user_reviews['reviews_year'].astype('int')

In [41]:
# Paso 3: Creación del DataFrame combinado y guardarlo como Parquet
combined_df = pd.merge(
    user_reviews[['reviews_item_id', 'reviews_year', 'reviews_recommend']],
    steam_games[['item_id', 'developer']].drop_duplicates(subset='item_id'),
    left_on='reviews_item_id', right_on='item_id'
)


In [42]:
combined_df

,reviews_item_id,reviews_year,reviews_recommend,item_id,developer
0,1250.0,2011,True,1250.0,Tripwire Interactive
1,1250.0,2015,True,1250.0,Tripwire Interactive
2,1250.0,2014,True,1250.0,Tripwire Interactive
3,1250.0,2013,True,1250.0,Tripwire Interactive
4,1250.0,2014,True,1250.0,Tripwire Interactive
...,...,...,...,...,...
43327,NaN,0,NaN,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
43328,NaN,0,NaN,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
43329,NaN,0,NaN,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
43330,NaN,0,NaN,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [43]:
combined_df.to_parquet('data/funcion_4.csv')

In [44]:
combined_df.to_parquet('data/funcion_4.parquet')

In [45]:
@app.get('/UsersWorstDeveloper')
def UsersWorstDeveloper(year: int):
    """
    Función para encontrar los tres desarrolladores de juegos con más reseñas negativas para un año dado.
    Esta función se basa en DataFrames pre-cargados: 'reviews_df' y 'games_df'.

    Parámetros:
    year (int): El año para el cual se buscan los desarrolladores.

    Devuelve:
    list: Los tres principales desarrolladores con más reseñas negativas para el año dado.
    """
    # Filtrar las reseñas para el año dado y donde la recomendación es False
    reviews_year = combined_df[(combined_df['reviews_year'] == year) & (combined_df['reviews_recommend'] == False)]

    # Si no hay reseñas negativas para ese año, retorna mensaje
    if reviews_year.empty:
        return f"No hay datos suficientes."

    # Contar el número de reseñas negativas para cada desarrollador
    developer_negative_reviews = reviews_year['developer'].value_counts()

    # Preparar la salida para los tres primeros desarrolladores, si hay menos de tres, maneja la situación
    top_developers = developer_negative_reviews.head(3)
    formatted_output = [{"Puesto {}".format(i + 1): dev} for i, dev in enumerate(top_developers.index)]
    
    return f"Las 3 desarrolladoras con juegos MENOS recomendados por usuarios para el año {year}: {formatted_output}"


In [46]:
UsersWorstDeveloper(2015)

"Las 3 desarrolladoras con juegos MENOS recomendados por usuarios para el año 2015: [{'Puesto 1': 'Valve'}, {'Puesto 2': 'Bohemia Interactive'}, {'Puesto 3': 'Facepunch Studios'}]"

## 5. Función def sentiment_analysis (empresa desarrolladora : str)

Para crear la función sentiment_analysis, seguiré estos pasos:

+ **Normalización:** Convertiré tanto el nombre de la desarrolladora proporcionado por el usuario como los nombres de las desarrolladoras en el conjunto de datos a minúsculas para garantizar la comparación sin diferencias de mayúsculas y minúsculas.

+ **Tolerancia a Errores de Escritura:** Para manejar errores menores en la escritura del nombre de la desarrolladora (como errores de ortografía), utilizaré una técnica de comparación de cadenas que permita cierto grado de variación. Una opción es utilizar la distancia de Levenshtein, que mide cuántas ediciones se necesitan para cambiar una cadena en otra.

+ **Búsqueda y Análisis:** La función buscará coincidencias en los nombres de las desarrolladoras, y luego realizará el análisis de sentimiento en las reseñas asociadas, agrupándolas en categorías de 'Negativo', 'Neutral' y 'Positivo'.

+ **Estructura de Resultados:** Finalmente, la función devolverá los resultados en el formato especificado.

**Creamos un dataframe ligero para la carga más rápida de la api**

In [47]:
# Seleccionar columnas relevantes
steam_games_light = steam_games[['item_id', 'developer']]
user_reviews_light = user_reviews[['reviews_item_id', 'sentiment_analysis']]

# Realizar un merge para combinar la información
combined_df_light = pd.merge(user_reviews_light, steam_games_light, 
                             left_on='reviews_item_id', right_on='item_id')

combined_df_light

,reviews_item_id,sentiment_analysis,item_id,developer
0,1250.0,2,1250.0,Tripwire Interactive
1,1250.0,0,1250.0,Tripwire Interactive
2,1250.0,2,1250.0,Tripwire Interactive
3,1250.0,2,1250.0,Tripwire Interactive
4,1250.0,2,1250.0,Tripwire Interactive
...,...,...,...,...
101277,NaN,1,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
101278,NaN,1,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
101279,NaN,1,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
101280,NaN,1,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


Revisamos cuántas valoraciones tiene el desarrollador "Tripwire Interactive" para luego comparar con el resultado con la función.

In [48]:
# Filtrar el DataFrame por el valor específico en la columna 'developer'
filtered_df = combined_df_light[combined_df_light['developer'] == "Tripwire Interactive"]

# Contar el número de filas en el DataFrame filtrado
count = len(filtered_df)

# Imprimir el resultado
print(f"Número de filas para el desarrollador 'Tripwire Interactive': {count}")

Número de filas para el desarrollador 'Tripwire Interactive': 502


In [49]:
# Guardar el DataFrame combinado ligero en formato Parquet
combined_df_light.to_parquet('data/funcion_5.parquet')

In [50]:
@app.get('/sentiment_analysis')
def sentiment_analysis(developer_name: str) -> str:
    """
    Analiza el sentimiento de las reseñas para juegos de una empresa desarrolladora específica.
    
    Args:
    developer_name (str): El nombre de la empresa desarrolladora. La función es insensible a mayúsculas/minúsculas.

    Returns:
    str: Un mensaje con el conteo de reseñas negativas, neutrales y positivas.
    """

def sentiment_analysis(developer_name: str) -> str:
    # Cargar el DataFrame ligero
    combined_df_light = pd.read_parquet('data/funcion_5.parquet')

    # Normalizar el nombre de la desarrolladora
    developer_name_normalized = developer_name.lower()

    # Filtrando juegos por la desarrolladora especificada
    filtered_reviews = combined_df_light[combined_df_light['developer'].str.lower() == developer_name_normalized]

    # Contando las reseñas por análisis de sentimiento
    sentiment_counts = filtered_reviews['sentiment_analysis'].value_counts().sort_index()
    sentiment_counts = { "Negative": sentiment_counts.get(0, 0), 
                         "Neutral": sentiment_counts.get(1, 0), 
                         "Positive": sentiment_counts.get(2, 0) }

    # Formateando el mensaje de salida
    return f"El análisis de sentimiento de reseñas para la desarrolladora {developer_name} es: [Negative = {sentiment_counts['Negative']}, Neutral = {sentiment_counts['Neutral']}, Positive = {sentiment_counts['Positive']}]"


In [51]:
sentiment_analysis("Tripwire Interactive") 

'El análisis de sentimiento de reseñas para la desarrolladora Tripwire Interactive es: [Negative = 72, Neutral = 164, Positive = 266]'